In [32]:
#Step1. rakutenからメニューデータを集める
import pandas as pd
from recommend_recipe import fetch_rakuten_recipes
CSV_PATH = './data/rakuten_recipe_category.csv'
df_keyword = pd.read_csv(CSV_PATH)
print(len(df_keyword))
SAVE_PATH = './data/all_df_recipe.csv'

all_df_recipe = fetch_rakuten_recipes(df_keyword[0:1]) #はじめに形を決定する
for i in range(1,len(df_keyword)):
    df_recipe = fetch_rakuten_recipes(df_keyword[i:i+1])

    all_df_recipe = pd.concat([all_df_recipe,df_recipe], ignore_index=True) #ignore_index=Trueによりdfのindexをリセットできる

    if i%100==0:
        all_df_recipe.to_csv(SAVE_PATH, index=False)  # インデックスは保存しない
        print(f"Saved len:{i}")


2161
Saved len:100
Saved len:200
Saved len:300
Saved len:400
Saved len:500
Saved len:600
Saved len:700
Saved len:800
Saved len:900
Saved len:1000
Saved len:1100
Saved len:1200
Saved len:1300
Saved len:1400
Saved len:1500
Saved len:1600
Saved len:1700
Saved len:1800
Saved len:1900
Saved len:2000
Saved len:2100


In [15]:
#Step2. レシピurlを読み出し＆レシピhtmlを取得して保存する
import pandas as pd
import urllib.request
import time
import pickle
import os
SAVE_PATH = './data/all_df_recipe.csv'
df_loaded = pd.read_csv(SAVE_PATH)
saved_html_list = os.listdir("./data/recipe_htmls/")

for recipeId,recipeTitle,recipeUrl in zip(df_loaded["recipeId"],df_loaded["recipeTitle"],df_loaded["recipeUrl"]):
    if f"{recipeId}.html" in saved_html_list:
        continue
    
    try:
        recipe_html = urllib.request.urlopen(recipeUrl).read().decode('utf-8')#htmlを取得する

        with open("./data/recipe_htmls/" + f"{recipeId}.html",mode="w") as f:
            f.write(recipe_html)
        
        time.sleep(2) #一応良心あるんで・・・
    except Exception as e:
        print(e)

HTTP Error 500: 500


In [2]:
#Step3.  保存したhtmlをLLMで読み取ってグラフを作成する
from generate_graph_local import Graph_Generator
graph_generator = Graph_Generator(USE_LOCAL_LLM=False)

In [3]:
# graph_generator.USE_LOCAL_LLM = True
# url = "https://recipe.rakuten.co.jp/recipe/1290007270/?l-id=wi_rcp_Taboola"
# graph = graph_generator.generate_graph(url)
# print(graph)

from bs4 import  BeautifulSoup
import os
# GPT_messages_outputs = os.listdir(f"./data/GPT_messages_outputs")

saved_html_list = os.listdir("./data/recipe_htmls/")
len(saved_html_list)
for html in saved_html_list[101:300]:
    html = f"./data/recipe_htmls/{html}"
    with open(html,"r",encoding="utf-8") as f:
        graph = graph_generator.generate_graph(url=None,html=f,save_message=True) #ここで出力をセーブするか決定する
        print(graph)

text_content タイトル: ひな祭り♪３色ミルクプリン レシピ・作り方

材料:
粉ゼラチン 5g
牛乳 300cc弱
スティック抹茶オレ 1本
いちごジャム 大さじ2
湯 大さじ3

手順:
レンジOKの容器にお湯を入れゼラチンを振り入れ少しふやかしておく。
1をレンジ（500w）で40秒チン。かき混ぜてゼラチンを溶かす。溶け残るなら20秒づつ位繰り返し溶かす。ほんの少々の溶け残りには目を瞑りまーす
合わせて300cc位になるように牛乳を加え混ぜる。100ccづつスティック抹茶、いちごジャムを溶かす。このまま飲んでも美味しそう（＾u＾）
瓶やグラスを鍋等に置き氷と水を張る。ザル等で漉しながら先ず抹茶味を注ぐ。
15分程でだいたい固まっていたら次はミルク味をそっと漉しながら注ぐ。イチゴ味も同様に繰り返す。今回は果肉入りジャムだったのでザルで濾すのはやめました。
蓋やラップをして冷蔵庫で2時間程冷やせば完成◎


output_str:
 Knowledge Graph:

Node 1: 粉ゼラチン
Type: ingredient
Quantity: 5g

Node 2: 牛乳
Type: ingredient
Quantity: 300cc弱

Node 3: スティック抹茶オレ
Type: ingredient
Quantity: 1本

Node 4: いちごジャム
Type: ingredient
Quantity: 大さじ2

Node 5: 湯
Type: ingredient
Quantity: 大さじ3

Node 6: ゼラチン液
Type: intermediate

Node 7: 抹茶ミルク
Type: intermediate

Node 8: イチゴミルク
Type: intermediate

Node 9: ミルクプリン
Type: final

Edge 1: 粉ゼラチン - ゼラチン液（振り入れる）
Edge 2: 湯 - ゼラチン液（入れる）
Edge 3: ゼラチン液 - 抹茶ミルク（混ぜる）
Edge 4: ゼラチン液 - イチゴミルク（混ぜる）
Edge 5: 牛乳 - 抹茶ミルク（加える）
Edge 6: 牛乳 - イチゴミルク（加える）
Edge 7: スティック抹茶オレ - 抹茶ミルク（溶かす）
Edge 8: いちごジャム - イチゴミルク（溶かす）
Edge 9: 抹茶ミルク - ミルクプリン（注ぐ）
Edge 10: イチゴミルク - ミルクプリン（注ぐ）
output_json: {'nodes': [{'id': '粉ゼラチン', 'type': 'ingredient', 'quantity': '5g'}, {'id': '牛乳', 'type': 'ingredient', 'quantity': '300cc弱'}, {'id': 'スティック抹茶オレ', 'type': 'ingredient', 'quantity': '1本'}, {'id': 'いちごジャム', 'type': 'ingredient', 'quantity': '大さじ2'}, {'id': '湯', 'type': 'ingredient', 'quantity': '大さじ3'}, {'id': 'ゼラチン液', 'typ

In [ ]:
#Step4. データセットを読み込んでtokenizerでプロンプトに変換する
import json
import os
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
from datasets import load_dataset,Dataset

# model_name = "elyza/Llama-3-ELYZA-JP-8B"
model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

GPT_messages_outputs = os.listdir(f"./data/GPT_messages_outputs")

custom_dataset = {"text":[]}
for _json in GPT_messages_outputs:
    _json = f"./data/GPT_messages_outputs/{_json}"
    with open(_json,"r",encoding="utf-8") as f:
        d = json.load(f)
        messages = d["messages"]
        messages.append({'role': 'assistant', 'content': d["output_str"]})

        # print(messages)

        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            # add_generation_prompt=True #これは初期状態で適用されている？
        )
        custom_dataset["text"].append(prompt)
        # print(prompt)

custom_dataset = Dataset.from_dict(custom_dataset) #transformersで扱えるDataset形式に変換する
custom_dataset
custom_dataset[93]["text"]

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'<|system|>\nあなたは以下の例のように料理のレシピを知識グラフに変換してください。<|end|>\n<|system|>\n【例1】<|end|>\n<|system|>\nタイトル\nご飯が進む！豚肉の生姜焼き\n\n材料(2人前)\n豚ロース (薄切り)\n300g\n薄力粉\n大さじ1\nたれ\nすりおろし生姜\n大さじ2\nしょうゆ\n大さじ2\n砂糖\n大さじ1\n料理酒\n大さじ1\nごま油\n大さじ1\n付け合わせ\nキャベツ\n\n手順\n準備.\nキャベツは芯を切り落としておきます。\n1.\nキャベツは千切りにします。\n2.\n豚ロースに薄力粉をまぶします。\n3.\nボウルにたれの材料を入れて混ぜ合わせます。\n4.\n中火で熱したフライパンにごま油をひき、2を入れて焼きます。豚ロースに火が通り、両面にこんがりと焼き色が付いたら3を入れます。\n5.\n中火で炒め合わせ、全体に味がなじんだら火から下ろします。\n6.\nお皿に1、5を盛り付けて完成です。\n<|end|>\n<|system|>\n{"nodes": [{"id": "\\u8c5a\\u30ed\\u30fc\\u30b9", "type": "ingredient", "quantity": "300g"}, {"id": "\\u8584\\u529b\\u7c89", "type": "ingredient", "quantity": "\\u5927\\u3055\\u30581"}, {"id": "\\u3059\\u308a\\u304a\\u308d\\u3057\\u751f\\u59dc", "type": "ingredient", "quantity": "\\u5927\\u3055\\u30582"}, {"id": "\\u3057\\u3087\\u3046\\u3086", "type": "ingredient", "quantity": "\\u5927\\u3055\\u30582"}, {"id": "\\u7802\\u7cd6", "type": "ingredient", "quantity": "\\u5927\\u3055\\u30581"}, {"id": "\\u6599\\u7406\\u9152", "

In [ ]:
#Step5. LoRAを実行して学習を行う
# https://highreso.jp/edgehub/machinelearning/llama3sftqlora.html
#上記のサイトが参考になりました
# https://highreso.jp/edgehub/machinelearning/wandbstartup.html
#↑　wanddbの使い方

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #これによりgpu1がcuda:0として扱われるようになるらしい、複数台利用したいときは0,1などにすればよい

from peft import LoraConfig, PeftModel
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

#  pip install trl==0.12.0 このversionがいいらしい
from trl import SFTTrainer
import wandb

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=["embed_tokens"],
)

training_arguments = TrainingArguments(
    output_dir="./results",
    bf16=True,
    per_device_train_batch_size=4, #このバッチサイズの調整でVRAMの量を変更できる,めちゃくちゃギリギリだけど48GBでbatch4が動く
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    optim="adamw_torch_fused",
    learning_rate=2e-4, 
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=10,
    group_by_length=True,
    report_to="wandb" 
)

new_model="Phi-3.5-mini-instruct-lora_ft"

trainer = SFTTrainer(
    model=model_name,
    tokenizer=tokenizer,
    train_dataset=custom_dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    args=training_arguments,
    max_seq_length=1024,
    packing=True,
)

wandb.init(project=new_model)
trainer.train()
trainer.model.save_pretrained(new_model)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:202: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a mo

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.041000
20,0.990600
30,0.865200
40,0.716800
50,0.621000
60,0.487200
70,0.360500
80,0.254500
90,0.191200
100,0.160800


In [ ]:
#ロードできるかの確認

# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     BitsAndBytesConfig,
#     HfArgumentParser,
#     TrainingArguments,
#     pipeline,
#     logging
# )

# import torch
# torch.cuda.empty_cache()

# new_model=f"Phi-3.5-mini-instruct-lora_ft"
# ft_model = AutoModelForCausalLM.from_pretrained(
#     new_model, 
#     device_map='auto',
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2"
# )

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


In [ ]:
#学習済みモデルを利用してみる
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #これによりgpu1がcuda:0として扱われるようになるらしい、複数台利用したいときは0,1などにすればよい
from generate_graph_local import Graph_Generator
graph_generator = Graph_Generator(USE_LOCAL_LLM=True,model_name="Phi-3.5-mini-instruct-lora_ft",tokenizer_name ="microsoft/Phi-3.5-mini-instruct" )

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.22it/s]


In [2]:
from bs4 import  BeautifulSoup
import os


saved_html_list = os.listdir("./data/recipe_htmls/")
len(saved_html_list)
for html in saved_html_list[4:100]:
    html = f"./data/recipe_htmls/{html}"
    with open(html,"r",encoding="utf-8") as f:
        graph = graph_generator.generate_graph(url=None,html=f)
        print(graph)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


text_content タイトル: 【ジャム】いちごジャムのアクでいちごみるく レシピ・作り方

材料:
いちごジャムのアク 大さじ1杯
ジャムが完成した後の鍋についてるジャム 大さじ1杯
牛乳 200㎖

手順:
【いちごジャム】いちごジャムを作る時にアクがたくさんでます！1回目に煮詰めた時のアクをしっかり取り除く
【煮詰める】煮詰めた時のアクもこまめにとるいちごジャムのアクはしっかり取ることで仕上がりが綺麗です！
【とれたアク】とれたアクを集めるジャムが完成した時の鍋に適量の牛乳を入れて鍋についてるジャムをとる
【完成】大さじ1~2杯でそのままいちごミルクとして楽しめます☆作るいちごジャムの量で取れるアクの量が変わるので少ない時は作ったジャムで調整してください☆
output_str:
 Knowledge Graph:

Node 1: いちごジャムのアク
Type: ingredient
Quantity: 大さじ1杯

Node 2: ジャムが完成した後の鍋についてるジャム
Type: ingredient
Quantity: 大さじ1杯

Node 3: 牛乳
Type: ingredient
Quantity: 200㎖

Node 4: いちごジャム
Type: intermediate

Node 5: ミルクジャム
Type: final

Edge 1: いちごジャムのアク - いちごジャム（浸す）
Edge 2: 煮詰めた時のアク - いちごジャム（取る）
Node 3: 牛乳 - ミルクジャム（混ぜる）
Node 4: ジャムが完成した後の鍋についてるジャム - ミルクジャム（浸す）
Edge 5: 牛乳 - ミルクジャム（混ぜる）
Node 5: いちごジャム - ミルクジャム（混ぜる）
output_json: {'nodes': [{'id': 'いちごジャムのアク', 'type': 'ingredient', 'quantity': '大さじ1杯'}, {'id': 'ジャムが完成した後の鍋についてるジャム', 'type': 'ingredient', 'quantity': '大さじ1杯'}, {'id': '牛乳', 'type': 'ingredient', 'quantity': '200㎖'}, {'id': 'いち

KeyboardInterrupt: 